In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensornetworks_pytorch.TNModels import PosMPS, Born
print(torch.__version__)

1.8.0.dev20201128


Use some data

In [15]:
import pickle
for dataset in [#'biofam',
    'flare','lymphography','spect','tumor','votes']:
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0].astype(int)
    print(dataset)
    print("\tdata shape:", X.shape)
    print(f"\trange of X values: {X.min()} -- {X.max()}")

def load_dataset(dataset):
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0]
    X=X.astype(int)

    print("data shape:", X.shape)
    print(f"range of X values: {X.min()} -- {X.max()}")
    print(f"setting d={X.max()+1}")
    d = X.max()+1
    return X, d

flare
	data shape: (1065, 13)
	range of X values: 0 -- 7
lymphography
	data shape: (148, 19)
	range of X values: 0 -- 7
spect
	data shape: (187, 23)
	range of X values: 0 -- 1
tumor
	data shape: (339, 17)
	range of X values: 0 -- 3
votes
	data shape: (435, 17)
	range of X values: 0 -- 2


In [21]:
X,d = load_dataset('tumor')

data shape: (339, 17)
range of X values: 0 -- 3
setting d=4


In [23]:
D = 7
mps = PosMPS(D=D, d=d, verbose=True)
rBorn = Born(D=D, d=d, dtype=torch.float, verbose=True) 
cBorn = Born(D=D, d=d, dtype=torch.cfloat, verbose=True)
models = (mps, rBorn, cBorn)
for model in models:
    print(model.core.shape, model.name)
print("===")
for model in models:
    model.fit(X, d)

torch.Size([4, 7, 7]) Positive MPS
torch.Size([4, 7, 7]) Born model torch.float32
torch.Size([4, 7, 7]) Born model torch.complex64
===


In [25]:
batchsize=10
trainloader = DataLoader(X, batch_size=batchsize, shuffle=True)

model = rBorn
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for batch_idx, batch in enumerate(trainloader):
    model.zero_grad()
    neglogprob = 0
    for x in batch:
        neglogprob -= model(x)
    loss = neglogprob / batchsize
    loss.backward()
    print(batch_idx,len(batch), "\tavg neglogprob:", loss.item())
    optimizer.step()

0 10 	avg neglogprob: 12.669713973999023
1 10 	avg neglogprob: 13.751876831054688
2 10 	avg neglogprob: 13.196316719055176
3 10 	avg neglogprob: 13.420600891113281
4 10 	avg neglogprob: 13.215057373046875
5 10 	avg neglogprob: 13.33989429473877
6 10 	avg neglogprob: 12.457732200622559
7 10 	avg neglogprob: 12.091470718383789
8 10 	avg neglogprob: 13.230816841125488
9 10 	avg neglogprob: 11.981729507446289
10 10 	avg neglogprob: 14.369333267211914
11 10 	avg neglogprob: 13.473103523254395
12 10 	avg neglogprob: 13.077798843383789
13 10 	avg neglogprob: 13.565008163452148
14 10 	avg neglogprob: 12.476985931396484
15 10 	avg neglogprob: 13.912542343139648
16 10 	avg neglogprob: 13.332720756530762
17 10 	avg neglogprob: 12.860852241516113
18 10 	avg neglogprob: 12.136152267456055
19 10 	avg neglogprob: 13.606549263000488
20 10 	avg neglogprob: 14.618481636047363
21 10 	avg neglogprob: 13.349733352661133
22 10 	avg neglogprob: 13.284083366394043
23 10 	avg neglogprob: 14.018351554870605
24 